In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv(r"./Aleppo2017_processed.csv",encoding='utf-8')
df.head()

,id,time,gl
0,183,2015-05-16 05:35:41,162.0
1,183,2015-05-16 05:30:41,164.0
2,183,2015-05-16 05:25:41,168.0
3,183,2015-05-16 05:20:41,169.0
4,183,2015-05-16 05:15:41,170.0


In [2]:
ID = df['id'].value_counts().index
print(ID)

Index([263,  77, 193, 277, 229, 245, 155, 251, 111, 164,
       ...
       128, 264, 186, 162, 249,  39,  52, 266, 223, 289],
      dtype='int64', name='id', length=226)


In [3]:
import datetime
for i in ID[:40]:
    #获取每个被试的数据
    data = df[df['id'] == i].copy()
    
    #对每个被试的数据根据时间进行排序
    #1、先将时间列设成datetime类型
    data['time'] = pd.to_datetime(data['time']) 
    
    #2、根据时间排序
    data = data.sort_values(by='time')
    
    #3、统计每个被试每天中有多少血糖数据
        #3.1 提取时间的年月日信息, 一天中的血糖数据的年月日信息相同
    data['date'] = data['time'].dt.date
    
        #3.2 统计相同的年月日中有多少血糖数据
    daily_data_count = data.groupby('date')['gl'].count()
    
    #4、筛选数据
        #4.1 连续7天
        #4.2 每天的数据量大于260
    continuous_days = 0    #记录连续的天数
    target_dates = []    #存放筛选出的数据(time)
    
    #此段for循环找出参与者第一个连续30天，且每天血糖数据大于250的日期
    for date,count in daily_data_count.items():
        if continuous_days >= 30:
            break
        else:
            if count > 250:
                target_dates.append(date)
                continuous_days += 1
            else:
                continuous_days = 0
                target_dates = []

    #5、处理target_dates中记录的日期对应的数据
    #5.1 获取处理target_dates中每天的数据
    total_data = []
    
    if target_dates:
        
        for j in target_dates:

            data_of_day = data[data['date'] == j]

            #5.2 如果这天中的数据满足288条则直接写入csv,如果不满足288则需要进行插值处理
            if data_of_day.shape[0] == 288:

                print("{}这天测够了288条数据".format(j))

                data_of_day = data_of_day.set_index('time')

                total_data.append(data_of_day)

            else:
                #先创建一个空的dataframe，将每小时的数据拼接到空的dataframe

                columns_name = ['id','gl']

                day_of_data = pd.DataFrame(columns=columns_name)

                day_of_data.index.name = 'time'

                print("{}这天的血糖记录数不满足288,这天的数据shape为{}".format(j,data_of_day.shape))

                #设置秒为0
                data_of_day.loc[:,'time'] = data_of_day['time'].apply(lambda dt: dt.replace(second=0))

                for k in range(24):

                    hour_data = data_of_day[data_of_day['time'].dt.hour == k].copy()    #某些小时可能没有数据

                    if hour_data.empty:

                        print("{}这天中数据在 {}-{}  小时时间范围内为空".format(j, k, k+1))

                        start_date = datetime.datetime.combine(j, datetime.time(k, 0))

                        full_time_index = pd.date_range(start=start_date, periods=12, freq='5T')

                        data_empty = {'gl': np.nan * len(full_time_index),'id':[i] * len(full_time_index)}

                        data_day_padding = pd.DataFrame(data_empty, index=full_time_index)

                        day_of_data = pd.concat((day_of_data,data_day_padding),axis=0)

                    else:
                        minute = hour_data.iloc[0]['time'].minute % 5

                        # 设置一小时内的起始时间
                        start_date = hour_data.iloc[0]['time'].replace(hour=k, minute=minute)
                        #print(start_date)

                        # 使用完整的时间序列重新索引
                        full_time_index = pd.date_range(start=start_date, periods=12, freq='5T')

                        #删除重复时间的记录
                        data_unique_times = hour_data[~hour_data['time'].duplicated(keep='first')]

                        # 将时间戳设置为索引
                        data_unique_times = data_unique_times.set_index('time')

                        # 使用完整的时间序列重新索引
                        hour_data_interpolated = data_unique_times.reindex(index=full_time_index)

                        hour_data_interpolated['id'] = i

                        day_of_data = pd.concat((day_of_data, hour_data_interpolated),axis=0)

                total_data.append(day_of_data)

        # 将 list 转换成Dataframe(total_data_pd包括一个参与者30天的血糖数据)
        total_data_pd = pd.concat(total_data) 

        # 进行线性插值interpolate()
        total_data_pd['gl'] = total_data_pd['gl'].interpolate().round()

        total_data_pd = total_data_pd.fillna({'gl': 0})

        print("线性插值后的数据:", total_data_pd)

        total_data_pd.to_csv(r"C:\Users\admin\Desktop\week_of_data13.csv", mode='a', header=False,index=True)
        
    else:
        print("编号为{}的参与者找不到时间上连续（大于30天）且每天的血糖数据记录量大于250的日期".format(i))
        print("*" *50)

2015-09-18这天的血糖记录数不满足288,这天的数据shape为(289, 4)
2015-09-19这天的血糖记录数不满足288,这天的数据shape为(290, 4)
2015-09-20这天的血糖记录数不满足288,这天的数据shape为(290, 4)
2015-09-21这天的血糖记录数不满足288,这天的数据shape为(289, 4)
2015-09-22这天的血糖记录数不满足288,这天的数据shape为(290, 4)
2015-09-23这天测够了288条数据
2015-09-24这天的血糖记录数不满足288,这天的数据shape为(258, 4)
2015-09-24这天中数据在 9-10  小时时间范围内为空
2015-09-24这天中数据在 10-11  小时时间范围内为空
2015-09-25这天测够了288条数据
2015-09-26这天测够了288条数据
2015-09-27这天的血糖记录数不满足288,这天的数据shape为(290, 4)
2015-09-28这天的血糖记录数不满足288,这天的数据shape为(289, 4)
2015-09-29这天的血糖记录数不满足288,这天的数据shape为(289, 4)
2015-09-30这天测够了288条数据
2015-10-01这天的血糖记录数不满足288,这天的数据shape为(260, 4)
2015-10-01这天中数据在 10-11  小时时间范围内为空
2015-10-02这天的血糖记录数不满足288,这天的数据shape为(290, 4)
2015-10-03这天的血糖记录数不满足288,这天的数据shape为(290, 4)
2015-10-04这天的血糖记录数不满足288,这天的数据shape为(289, 4)
2015-10-05这天的血糖记录数不满足288,这天的数据shape为(290, 4)
2015-10-06这天的血糖记录数不满足288,这天的数据shape为(289, 4)
2015-10-07这天的血糖记录数不满足288,这天的数据shape为(290, 4)
2015-10-08这天的血糖记录数不满足288,这天的数据shape为(261, 4)
2015-10-08这天中数据在 8-9  小时时间范围内为空
2015-10-08这天中数

2015-06-15这天的血糖记录数不满足288,这天的数据shape为(289, 4)
2015-06-16这天测够了288条数据
2015-06-17这天的血糖记录数不满足288,这天的数据shape为(280, 4)
2015-06-18这天的血糖记录数不满足288,这天的数据shape为(291, 4)
2015-06-19这天的血糖记录数不满足288,这天的数据shape为(256, 4)
2015-06-19这天中数据在 9-10  小时时间范围内为空
2015-06-20这天的血糖记录数不满足288,这天的数据shape为(290, 4)
2015-06-21这天的血糖记录数不满足288,这天的数据shape为(289, 4)
2015-06-22这天的血糖记录数不满足288,这天的数据shape为(292, 4)
2015-06-23这天测够了288条数据
2015-06-24这天的血糖记录数不满足288,这天的数据shape为(289, 4)
2015-06-25这天的血糖记录数不满足288,这天的数据shape为(290, 4)
2015-06-26这天的血糖记录数不满足288,这天的数据shape为(255, 4)
2015-06-26这天中数据在 8-9  小时时间范围内为空
2015-06-26这天中数据在 9-10  小时时间范围内为空
2015-06-27这天的血糖记录数不满足288,这天的数据shape为(290, 4)
2015-06-28这天的血糖记录数不满足288,这天的数据shape为(293, 4)
2015-06-29这天测够了288条数据
2015-06-30这天测够了288条数据
2015-07-01这天的血糖记录数不满足288,这天的数据shape为(286, 4)
2015-07-02这天的血糖记录数不满足288,这天的数据shape为(280, 4)
2015-07-03这天的血糖记录数不满足288,这天的数据shape为(259, 4)
2015-07-03这天中数据在 9-10  小时时间范围内为空
2015-07-04这天的血糖记录数不满足288,这天的数据shape为(277, 4)
2015-07-05这天测够了288条数据
2015-07-06这天的血糖记录数不满足288,这天的数据shape为(27

2015-06-22这天的血糖记录数不满足288,这天的数据shape为(289, 4)
2015-06-23这天测够了288条数据
2015-06-24这天的血糖记录数不满足288,这天的数据shape为(290, 4)
2015-06-25这天测够了288条数据
2015-06-26这天的血糖记录数不满足288,这天的数据shape为(266, 4)
2015-06-26这天中数据在 8-9  小时时间范围内为空
2015-06-27这天的血糖记录数不满足288,这天的数据shape为(263, 4)
2015-06-27这天中数据在 13-14  小时时间范围内为空
2015-06-27这天中数据在 14-15  小时时间范围内为空
2015-06-28这天的血糖记录数不满足288,这天的数据shape为(286, 4)
2015-06-29这天的血糖记录数不满足288,这天的数据shape为(287, 4)
2015-06-30这天的血糖记录数不满足288,这天的数据shape为(293, 4)
2015-07-01这天的血糖记录数不满足288,这天的数据shape为(294, 4)
2015-07-02这天的血糖记录数不满足288,这天的数据shape为(258, 4)
2015-07-02这天中数据在 13-14  小时时间范围内为空
2015-07-03这天的血糖记录数不满足288,这天的数据shape为(290, 4)
2015-07-04这天的血糖记录数不满足288,这天的数据shape为(289, 4)
2015-07-05这天的血糖记录数不满足288,这天的数据shape为(292, 4)
2015-07-06这天的血糖记录数不满足288,这天的数据shape为(290, 4)
2015-07-07这天的血糖记录数不满足288,这天的数据shape为(262, 4)
2015-07-07这天中数据在 2-3  小时时间范围内为空
2015-07-08这天的血糖记录数不满足288,这天的数据shape为(286, 4)
2015-07-09这天的血糖记录数不满足288,这天的数据shape为(293, 4)
2015-07-10这天测够了288条数据
2015-07-11这天的血糖记录数不满足288,这天的数据shape为(289, 4)
201

2015-06-07这天的血糖记录数不满足288,这天的数据shape为(280, 4)
2015-06-08这天的血糖记录数不满足288,这天的数据shape为(290, 4)
2015-06-09这天的血糖记录数不满足288,这天的数据shape为(287, 4)
2015-06-10这天的血糖记录数不满足288,这天的数据shape为(259, 4)
2015-06-10这天中数据在 12-13  小时时间范围内为空
2015-06-11这天的血糖记录数不满足288,这天的数据shape为(291, 4)
2015-06-12这天的血糖记录数不满足288,这天的数据shape为(285, 4)
2015-06-13这天的血糖记录数不满足288,这天的数据shape为(282, 4)
2015-06-14这天的血糖记录数不满足288,这天的数据shape为(290, 4)
2015-06-15这天的血糖记录数不满足288,这天的数据shape为(287, 4)
2015-06-16这天的血糖记录数不满足288,这天的数据shape为(291, 4)
2015-06-17这天的血糖记录数不满足288,这天的数据shape为(264, 4)
2015-06-17这天中数据在 6-7  小时时间范围内为空
2015-06-18这天的血糖记录数不满足288,这天的数据shape为(287, 4)
2015-06-19这天测够了288条数据
2015-06-20这天的血糖记录数不满足288,这天的数据shape为(289, 4)
2015-06-21这天的血糖记录数不满足288,这天的数据shape为(279, 4)
2015-06-22这天的血糖记录数不满足288,这天的数据shape为(284, 4)
2015-06-23这天的血糖记录数不满足288,这天的数据shape为(286, 4)
2015-06-24这天的血糖记录数不满足288,这天的数据shape为(266, 4)
2015-06-24这天中数据在 6-7  小时时间范围内为空
2015-06-24这天中数据在 7-8  小时时间范围内为空
2015-06-25这天的血糖记录数不满足288,这天的数据shape为(291, 4)
2015-06-26这天的血糖记录数不满足288,这天的数据shape为(2

编号为67的参与者找不到时间上连续（大于30天）且每天的血糖数据记录量大于250的日期
**************************************************
2015-05-30这天的血糖记录数不满足288,这天的数据shape为(287, 4)
2015-05-31这天的血糖记录数不满足288,这天的数据shape为(287, 4)
2015-06-01这天测够了288条数据
2015-06-02这天的血糖记录数不满足288,这天的数据shape为(286, 4)
2015-06-03这天的血糖记录数不满足288,这天的数据shape为(291, 4)
2015-06-04这天的血糖记录数不满足288,这天的数据shape为(289, 4)
2015-06-05这天的血糖记录数不满足288,这天的数据shape为(260, 4)
2015-06-05这天中数据在 13-14  小时时间范围内为空
2015-06-05这天中数据在 14-15  小时时间范围内为空
2015-06-06这天的血糖记录数不满足288,这天的数据shape为(289, 4)
2015-06-07这天的血糖记录数不满足288,这天的数据shape为(291, 4)
2015-06-08这天测够了288条数据
2015-06-09这天的血糖记录数不满足288,这天的数据shape为(291, 4)
2015-06-10这天测够了288条数据
2015-06-11这天的血糖记录数不满足288,这天的数据shape为(287, 4)
2015-06-12这天的血糖记录数不满足288,这天的数据shape为(257, 4)
2015-06-12这天中数据在 13-14  小时时间范围内为空
2015-06-12这天中数据在 14-15  小时时间范围内为空
2015-06-12这天中数据在 15-16  小时时间范围内为空
2015-06-13这天测够了288条数据
2015-06-14这天测够了288条数据
2015-06-15这天的血糖记录数不满足288,这天的数据shape为(289, 4)
2015-06-16这天测够了288条数据
2015-06-17这天的血糖记录数不满足288,这天的数据shape为(291, 4)
2015-06-18这天的血糖记录数

编号为240的参与者找不到时间上连续（大于30天）且每天的血糖数据记录量大于250的日期
**************************************************
2015-07-04这天的血糖记录数不满足288,这天的数据shape为(291, 4)
2015-07-05这天的血糖记录数不满足288,这天的数据shape为(291, 4)
2015-07-06这天的血糖记录数不满足288,这天的数据shape为(290, 4)
2015-07-07这天的血糖记录数不满足288,这天的数据shape为(290, 4)
2015-07-08这天的血糖记录数不满足288,这天的数据shape为(290, 4)
2015-07-09这天的血糖记录数不满足288,这天的数据shape为(289, 4)
2015-07-10这天的血糖记录数不满足288,这天的数据shape为(264, 4)
2015-07-10这天中数据在 11-12  小时时间范围内为空
2015-07-11这天的血糖记录数不满足288,这天的数据shape为(291, 4)
2015-07-12这天的血糖记录数不满足288,这天的数据shape为(290, 4)
2015-07-13这天的血糖记录数不满足288,这天的数据shape为(289, 4)
2015-07-14这天的血糖记录数不满足288,这天的数据shape为(291, 4)
2015-07-15这天的血糖记录数不满足288,这天的数据shape为(290, 4)
2015-07-16这天的血糖记录数不满足288,这天的数据shape为(289, 4)
2015-07-17这天的血糖记录数不满足288,这天的数据shape为(260, 4)
2015-07-17这天中数据在 11-12  小时时间范围内为空
2015-07-18这天的血糖记录数不满足288,这天的数据shape为(289, 4)
2015-07-19这天的血糖记录数不满足288,这天的数据shape为(290, 4)
2015-07-20这天的血糖记录数不满足288,这天的数据shape为(284, 4)
2015-07-21这天的血糖记录数不满足288,这天的数据shape为(291, 4)
2015-07-22这天的血糖记录数不满足288,这

2015-06-13这天的血糖记录数不满足288,这天的数据shape为(292, 4)
2015-06-14这天的血糖记录数不满足288,这天的数据shape为(292, 4)
2015-06-15这天的血糖记录数不满足288,这天的数据shape为(291, 4)
2015-06-16这天的血糖记录数不满足288,这天的数据shape为(291, 4)
2015-06-17这天的血糖记录数不满足288,这天的数据shape为(291, 4)
2015-06-18这天的血糖记录数不满足288,这天的数据shape为(292, 4)
2015-06-19这天的血糖记录数不满足288,这天的数据shape为(261, 4)
2015-06-19这天中数据在 8-9  小时时间范围内为空
2015-06-19这天中数据在 9-10  小时时间范围内为空
2015-06-20这天的血糖记录数不满足288,这天的数据shape为(293, 4)
2015-06-21这天的血糖记录数不满足288,这天的数据shape为(291, 4)
2015-06-22这天的血糖记录数不满足288,这天的数据shape为(291, 4)
2015-06-23这天的血糖记录数不满足288,这天的数据shape为(292, 4)
2015-06-24这天的血糖记录数不满足288,这天的数据shape为(293, 4)
2015-06-25这天的血糖记录数不满足288,这天的数据shape为(293, 4)
2015-06-26这天的血糖记录数不满足288,这天的数据shape为(269, 4)
2015-06-26这天中数据在 8-9  小时时间范围内为空
2015-06-27这天的血糖记录数不满足288,这天的数据shape为(292, 4)
2015-06-28这天的血糖记录数不满足288,这天的数据shape为(292, 4)
2015-06-29这天的血糖记录数不满足288,这天的数据shape为(293, 4)
2015-06-30这天的血糖记录数不满足288,这天的数据shape为(290, 4)
2015-07-01这天的血糖记录数不满足288,这天的数据shape为(292, 4)
2015-07-02这天的血糖记录数不满足288,这天的数据shape为(295, 4)
201

2015-06-15这天的血糖记录数不满足288,这天的数据shape为(292, 4)
2015-06-16这天的血糖记录数不满足288,这天的数据shape为(293, 4)
2015-06-17这天的血糖记录数不满足288,这天的数据shape为(292, 4)
2015-06-18这天的血糖记录数不满足288,这天的数据shape为(292, 4)
2015-06-19这天的血糖记录数不满足288,这天的数据shape为(284, 4)
2015-06-20这天的血糖记录数不满足288,这天的数据shape为(292, 4)
2015-06-21这天的血糖记录数不满足288,这天的数据shape为(257, 4)
2015-06-21这天中数据在 9-10  小时时间范围内为空
2015-06-21这天中数据在 10-11  小时时间范围内为空
2015-06-22这天的血糖记录数不满足288,这天的数据shape为(291, 4)
2015-06-23这天的血糖记录数不满足288,这天的数据shape为(291, 4)
2015-06-24这天的血糖记录数不满足288,这天的数据shape为(292, 4)
2015-06-25这天的血糖记录数不满足288,这天的数据shape为(292, 4)
2015-06-26这天的血糖记录数不满足288,这天的数据shape为(285, 4)
2015-06-27这天的血糖记录数不满足288,这天的数据shape为(290, 4)
2015-06-28这天的血糖记录数不满足288,这天的数据shape为(261, 4)
2015-06-28这天中数据在 10-11  小时时间范围内为空
2015-06-29这天的血糖记录数不满足288,这天的数据shape为(291, 4)
2015-06-30这天的血糖记录数不满足288,这天的数据shape为(291, 4)
2015-07-01这天的血糖记录数不满足288,这天的数据shape为(293, 4)
2015-07-02这天测够了288条数据
2015-07-03这天的血糖记录数不满足288,这天的数据shape为(282, 4)
2015-07-04这天的血糖记录数不满足288,这天的数据shape为(293, 4)
2015-07-05这天的血糖记录数不满足2

In [4]:
df10 = pd.read_csv(r"C:\Users\admin\Desktop\week_of_data13.csv",encoding='utf-8')

In [5]:
df10.columns = ['time', 'id', 'gl', 'date']

In [6]:
idx = df10['id'].value_counts().index
len(idx)

29

In [7]:
df10

,time,id,gl,date
0,2015-09-18 00:08:00,263,92.0,2015-09-18
1,2015-09-18 00:13:00,263,92.0,2015-09-18
2,2015-09-18 00:18:00,263,93.0,2015-09-18
3,2015-09-18 00:23:00,263,93.0,2015-09-18
4,2015-09-18 00:28:00,263,94.0,2015-09-18
...,...,...,...,...
250554,2015-07-14 23:38:00,198,149.0,2015-07-14
250555,2015-07-14 23:43:00,198,151.0,2015-07-14
250556,2015-07-14 23:48:00,198,151.0,2015-07-14
250557,2015-07-14 23:53:00,198,150.0,2015-07-14


In [8]:
250560 / 288

870.0